# Figure 1: Analyze Phospho Results for MAPK Signaling

In [2]:
import pandas as pd

import cptac
import cptac.utils as u
import plot_utils as p

import warnings
warnings.filterwarnings('ignore')

In [ ]:
'''
This function returns the color used in Figure 1 for every 
differential expression/p-value status of a certain phosphosite.
@Param site: String. Name of phosphosite column.
@Param df: df with pval and differential expressions. nan dropped.
@Param cancer: String. Cancer name used in df columns. 
'''

def check_status_color(site, df, cancer):
    df = df.loc[df['Phospho'] == site]
    pval = float(df[cancer+'_P_Value'])
    dif_expression = float(df[cancer+'_Median'])
    color = ''
    if pval >= 0.1:
        color += 'grey'
    else:
        if pval < .05:
            color += 'dark_'
        if dif_expression < 0:
            color += 'blue'
        elif dif_expression > 0:
            color += 'red'
        
    return color
    

# Step 1: Get tables with MAPK signaling phosphoproteomics.

In [6]:
df = pd.read_csv("../Make_Tables/csv/phospho_MAPK.csv")
luad_df = pd.read_csv("../Make_Tables/csv/Single_Cancer/Luad_phospho_MAPK.csv")
e_df = pd.read_csv("../Make_Tables/csv/Single_Cancer/Endo_phospho_MAPK.csv")
c_df = pd.read_csv("../Make_Tables/csv/Single_Cancer/Colon_phospho_MAPK.csv")

# Step 2: Create df with all phosphosites mapped with colors


Color Legend:
* red - differential expression > 0
* blue - differential expression < 0
* grey - not significant
* white - no data / not enough data for t-test

All sites used in Figure 1 have at least 1 FDR corrected p-value < 0.1. The darker hue of a color represents a  p-value < 0.05 (dark_red, dark_blue). 

In [12]:
e_df_2 = e_df.dropna() # drop nan (p-vals nan when not enough data to do t-test)
e_df_2['color'] = e_df_2['Phospho'].apply(lambda x: check_status_color(x, e_df_2, 'Endo'))
#e_df_2

In [11]:
luad_df_2 = luad_df.dropna()
luad_df_2['color'] = luad_df_2['Phospho'].apply(lambda x: check_status_color(x, luad_df_2, 'Luad'))
#luad_df_2

# Step 3: Show data for phosphosites used in KRAS Figure 

* Shown phosphosites are significant in at least 1 cancer.

In [13]:
# Get list of phosphosites sig in at least 1 cancer
e = e_df.loc[e_df['Endo_P_Value'] < 0.1]
e_sig_phospho = list(e.Phospho)

luad = luad_df.loc[luad_df['Luad_P_Value'] < 0.1]
luad_sig_phospho = list(luad.Phospho)

fig_phospho = list(set(luad_sig_phospho + e_sig_phospho))

In [17]:
combined = e_df_2.merge(luad_df_2, on = 'Phospho', how = 'outer', suffixes = ['_Endo', '_Luad'])
fig_data = combined[combined['Phospho'].isin(fig_phospho)]
fig_data.color_Luad = fig_data.color_Luad.fillna('white')
fig_data.color_Endo = fig_data.color_Endo.fillna('white')
fig_data

,Phospho,Endo_P_Value,Endo_Median,color_Endo,Luad_P_Value,Luad_Median,color_Luad
0,MKNK2_S220,0.001348,0.94700,dark_red,0.015097,1.33870,dark_red
1,NFKB2_S858,0.001348,0.69285,dark_red,0.015097,1.40475,dark_red
2,RAF1_T330,0.002399,1.12066,dark_red,0.118795,0.97875,grey
3,TAB1_S378,0.002916,0.46350,dark_red,NaN,NaN,white
4,MKNK1_S401,0.003520,0.47800,dark_red,0.424119,0.20325,grey
...,...,...,...,...,...,...,...
864,PLA2G4A_S434S435S437,NaN,NaN,white,0.082020,0.89615,red
865,FLNB_S2114,NaN,NaN,white,0.082020,2.17800,red
866,PLA2G4A_S435S437,NaN,NaN,white,0.085660,0.99990,red
867,NFKB2_S161,NaN,NaN,white,0.085660,0.79615,red


# Check that Colon has measurements in the MAPK pathway

In [18]:
# does colon have data for proteins sig in other cancers?
e_list = list(e_sig_2.Protein)
luad_list = list(luad_sig_2.Protein)
prot = list(set(e_list + luad_list))
print('Num proteins with sig sites in endo or luad:', len(prot), 'proteins')

prot_sig = c_df[c_df.Protein.isin(prot)].dropna()
print(len(prot_sig.Protein.value_counts()), 'of those proteins found in colon')


AttributeError: 'DataFrame' object has no attribute 'Protein'

In [19]:
# does colon have data for sites sig in other cancers?
e_list = list(e_sig_2.Phospho)
luad_list = list(luad_sig_2.Phospho)
sites = list(set(e_list + luad_list))
print('Num sig sites in endo or luad:', len(sites) ,'sites')

sites_sig = c_df[c_df.Phospho.isin(sites)]
print(len(sites_sig.Phospho.value_counts()), 'found in colon')

Num sig sites in endo or luad: 62 sites
0 found in colon
